# B92 Protocol

### Import all the needed packages

In [117]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
import pandas as pd

### Define all the meta parameters we will need for the protocol
We define the number of bits that are to be used in the circuit as n. For testing puposes, we can set the randomness seed to be the same every time.

In [118]:
n = 30
np.random.seed(seed=0)
test = True

### Create the qubits that Alice will send across the public channel

In [119]:
def alice_encode(input_arr):
    output = []
    for bit in input_arr:
        qc = QuantumCircuit(1,1)
        
        if bit == 1:
            qc.h(0)
        qc.barrier()
        output.append(qc)
    return output
            
            

In [120]:
alice_bits = randint(2, size=n)
alice_qubits = alice_encode(alice_bits)


if test:
    
    print('Alice bits:')
    print(alice_bits)
    
#     print('Alice qubits:')
#     for i in alice_qubits:
#         print(i)


Alice bits:
[0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 1 0]


### Bob decides generates an n-bit rnadom string and the measures Alices qubits.

In [121]:
def bob_decode(alice_qubits, bob_bits):
    output = []
    for alice_qubit, bob_bit in zip(alice_qubits, bob_bits):
        
        # Measure in the x-basis
        if bob_bit == 0:
            alice_qubit.h(0)
            alice_qubit.measure(0, 0)
        
        # Measure in the z-basis
        else:
            alice_qubit.measure(0, 0)
            
        # Simulate the measurement (taken directly from the qiskit example)
        qasm_sim = Aer.get_backend('qasm_simulator')
        qobj = assemble(alice_qubit, shots=1, memory=True)
        result = qasm_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        output.append(measured_bit)
    return output

def remove_zeros(alice_bits, bob_bits, bob_yield):
    bob_key = ''
    alice_key = ''
    
    for a_bit, b_bit, b_yield in zip(alice_bits, bob_bits, bob_yield):
        if b_yield == 1:
            bob_key += str(b_bit)
            alice_key += str(b_bit)

    return alice_key, bob_key
    

In [122]:
bob_bits = randint(2, size=n)
bob_yield = bob_decode(alice_qubits, bob_bits)
alice_key, bob_key = remove_zeros(alice_bits, bob_bits, bob_yield)

if test:
    print('Bob bits:')
    print(bob_bits)
    
    print('Bob yeild')
    print(bob_yield)
    
    print('Key stirings')
    print(f'Alice key: {alice_key}')
    print(f'Bob key: {bob_key}')

Bob bits:
[1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 0 1]
Bob yeild
[0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Key stirings
Alice key: 1011101
Bob key: 1011101
